In [2]:
!echo $PATH

/work/hpc/miniconda3/envs/layout/bin:/home/hpc/.vscode-server/cli/servers/Stable-6f17636121051a53c88d3e605c491d22af2ba755/server/bin/remote-cli:/home/hpc/.local/bin:/work/hpc/miniconda3/envs/layout/bin:/opt/hpc/openmpi/bin:/opt/hpc/ucx/bin:/opt/hpc/pmix/bin:/opt/hpc/munge/bin:/home/hpc/.local/bin:/usr/local/Modules/bin:/opt/hpc/openmpi/bin:/opt/hpc/ucx/bin:/opt/hpc/pmix/bin:/opt/hpc/munge/bin:/opt/conda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/snap/bin


In [3]:
%env PATH=/work/hpc/miniconda3/envs/layout/lib/python3.10/site-packages/tensorrt_libs:/work/hpc/miniconda3/envs/layout/lib/python3.10/site-packages/tensorrt_libs/libnvinfer.so.10:/home/hpc/.vscode-server/cli/servers/Stable-360a4e4fd251bfce169a4ddf857c7d25d1ad40da/server/bin/remote-cli:/home/hpc/.local/bin:/work/hpc/miniconda3/envs/layout/bin:/usr/local/Modules/bin:/opt/hpc/openmpi/bin:/opt/hpc/ucx/bin:/opt/hpc/pmix/bin:/opt/hpc/munge/bin:/opt/conda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/snap/bin:/home/hpc/.vscode-server/extensions/ms-python.debugpy-2025.10.0-linux-x64/bundled/scripts/noConfigScripts:/home/hpc/.vscode-server/data/User/globalStorage/github.copilot-chat/debugCommand

env: PATH=/work/hpc/miniconda3/envs/layout/lib/python3.10/site-packages/tensorrt_libs:/work/hpc/miniconda3/envs/layout/lib/python3.10/site-packages/tensorrt_libs/libnvinfer.so.10:/home/hpc/.vscode-server/cli/servers/Stable-360a4e4fd251bfce169a4ddf857c7d25d1ad40da/server/bin/remote-cli:/home/hpc/.local/bin:/work/hpc/miniconda3/envs/layout/bin:/usr/local/Modules/bin:/opt/hpc/openmpi/bin:/opt/hpc/ucx/bin:/opt/hpc/pmix/bin:/opt/hpc/munge/bin:/opt/conda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/snap/bin:/home/hpc/.vscode-server/extensions/ms-python.debugpy-2025.10.0-linux-x64/bundled/scripts/noConfigScripts:/home/hpc/.vscode-server/data/User/globalStorage/github.copilot-chat/debugCommand


In [4]:
import onnxruntime as ort

# All providers supported by your installed package
print("Available providers:", ort.get_available_providers())
# model = ort.InferenceSession("/home/hpc/.cache/huggingface/hub/models--unstructuredio--yolo_x_layout/snapshots/7680d6f857780bcf8d49916aa2e8881bd49dee3e/yolox_l0.05.onnx", providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider'], )

Available providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [5]:
import os
from unstructured.partition.auto import partition
from unstructured.staging.base import elements_to_json
from unstructured_inference.models.base import get_model, get_default_model_mappings
from unstructured_inference.inference.layout import DocumentLayout
from unstructured.partition.utils.constants import *
from bs4 import BeautifulSoup
import unicodedata
import re
import cv2
from copy import deepcopy
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import patches
from huggingface_hub import hf_hub_download


/work/hpc/miniconda3/envs/layout/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
%cd /data/hpc/potato/ecoit_summer/

%env OCR_LAYOUT_SUBREGION_THRESHOLD=0.7
%env EXTRACT_TABLE_AS_CELLS=True
%env TABLE_CLEAN_TEXT=True
# %env ECOIT_PARSE_LINE=False
%env DOCTR_MODEL_NAME=linknet_resnet34
%env DOCTR_THRESHOLD=0.0
%env DOCTR_DEVICE=cuda:0
%env DOCTR_DET_CLUSTER=TRUE

%env VIETOCR_MODEL_NAME=vgg_transformer
%env VIETOCR_DEVICE=cuda:0
%env VIETOCR_BATCH_SIZE=32
%env VIETOCR_MAX_TEXT_WIDTH_RATIO=0.5
%env ECOIT_TEXT_SPACING_RATIO=0.2
%env VIETOCR_USE_BEAM_SEARCH=False

/data/hpc/potato/ecoit_summer
env: OCR_LAYOUT_SUBREGION_THRESHOLD=0.7
env: EXTRACT_TABLE_AS_CELLS=True
env: TABLE_CLEAN_TEXT=True
env: DOCTR_MODEL_NAME=linknet_resnet34
env: DOCTR_THRESHOLD=0.0
env: DOCTR_DEVICE=cuda:0
env: DOCTR_DET_CLUSTER=TRUE
env: VIETOCR_MODEL_NAME=vgg_transformer
env: VIETOCR_DEVICE=cuda:0
env: VIETOCR_BATCH_SIZE=32
env: VIETOCR_MAX_TEXT_WIDTH_RATIO=0.5
env: ECOIT_TEXT_SPACING_RATIO=0.2
env: VIETOCR_USE_BEAM_SEARCH=False


# Test API 

In [10]:
def clean_text(text):
    """
    Remove all hyperlinks from the input text, keeping only the visible text.
    Supports formats like <a href="...">text</a> and raw URLs.
    """
    # Remove HTML anchor tags but keep the text inside
    text = re.sub(r'<a [^>]*>(.*?)</a>', r'\1', text, flags=re.IGNORECASE)
    # Remove raw URLs (http, https, www)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = unicodedata.normalize("NFKD", text)
    return text.strip()

In [26]:
index = 1
path = r"data/database/Báo cáo/test_RMU.pdf"
# path = r"data/database/07_KT/2024.TTr phê duyệt TCKT Flycam.pdf"

In [27]:
strategy = "hi_res" # Strategy for analyzing PDFs and extracting table structure
model_name = "yolox_quantized" # Best model for table extraction. Other options are detectron2_onnx and cheaper depending on file layout

# Extracts the elements from the PDF
elements = partition(
        filename=path, 
        strategy=strategy,
        languages=["vie", "eng"],
        infer_table_structure=True,
        skip_infer_table_types = False,
        hi_res_model_name=model_name,
        pdf_image_dpi=300,
        ocr_agent=OCR_AGENT_ECOIT,
        table_ocr_agent=OCR_AGENT_ECOIT
        )

# model = get_model("yolox")
# layout = DocumentLayout.from_file(path, detection_model=model, pdf_image_dpi=200)

Using unstructured.partition.utils.ocr_models.ecoit_ocr.OCRAgentECOIT model for extraction and unstructured.partition.utils.ocr_models.ecoit_ocr.OCRAgentECOIT for table extraction
hi_res


Processing page 70: 100%|██████████| 71/71 [07:56<00:00,  6.71s/it]


In [28]:
with open("logs/test/elements.txt", "w") as file:
    file.write("\n".join([el.metadata.ocr_text if el.metadata.ocr_text else el.text for el in elements if el.category != "Table" ]))

In [14]:
# example_page = layout.pages[5]
# image = example_page._get_image(path, 6, 200)
# layout_els = example_page.elements

In [15]:
# image.save("./logs/example.png")

In [16]:
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.imshow(image)
# for el in layout_els: 
#     if el.bbox is not None:
#         ax.add_patch(patches.Rectangle((el.bbox.x1, el.bbox.y1), el.bbox.x2 - el.bbox.x1, el.bbox.y2 - el.bbox.y1, fill=False, edgecolor='red', linewidth=2))
# plt.show()

In [17]:
# filepath = hf_hub_download(repo_id="juliozhao/DocLayout-YOLO-DocStructBench", filename="doclayout_yolo_docstructbench_imgsz1024.pt")
# doclayout_model = YOLOv10(filepath)

In [18]:
# type(doclayout_model)

In [19]:
# det_res = doclayout_model.predict(image, device="cuda:0", verbose=True)

In [20]:
# annotated_frame = det_res[0].plot(pil=True, line_width=5, font_size=20)
# cv2.imwrite("logs/result.jpg", annotated_frame)

In [17]:
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.imshow(image)
# for bbox in det_res.element_coords: 
#     if bbox is not None:
#         x1, y1, x2, y2 = bbox
#         ax.add_patch(patches.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2))
# plt.show()

In [29]:
tables = [el for el in elements if el.category == "Table"]

In [30]:
test = tables[0]

In [31]:
test.metadata.table_as_cells

[{'x': 0, 'y': 0, 'w': 1, 'h': 1, 'content': 'un'},
 {'x': 0, 'y': 1, 'w': 1, 'h': 1, 'content': 'KV'},
 {'x': 0, 'y': 2, 'w': 1, 'h': 1, 'content': ''},
 {'x': 0, 'y': 3, 'w': 1, 'h': 1, 'content': ''},
 {'x': 0, 'y': 4, 'w': 1, 'h': 1, 'content': ''},
 {'x': 0, 'y': 5, 'w': 1, 'h': 1, 'content': ''},
 {'x': 1, 'y': 0, 'w': 1, 'h': 1, 'content': 'In'},
 {'x': 1, 'y': 1, 'w': 1, 'h': 1, 'content': '25 31,5'},
 {'x': 1, 'y': 2, 'w': 1, 'h': 1, 'content': '40'},
 {'x': 1, 'y': 3, 'w': 1, 'h': 1, 'content': '50'},
 {'x': 1, 'y': 4, 'w': 1, 'h': 1, 'content': '80'},
 {'x': 1, 'y': 5, 'w': 1, 'h': 1, 'content': '100'},
 {'x': 2, 'y': 0, 'w': 1, 'h': 1, 'content': ''},
 {'x': 2, 'y': 1, 'w': 1, 'h': 1, 'content': '442 442'},
 {'x': 2, 'y': 2, 'w': 1, 'h': 1, 'content': '442'},
 {'x': 2, 'y': 3, 'w': 1, 'h': 1, 'content': '442'},
 {'x': 2, 'y': 4, 'w': 1, 'h': 1, 'content': '442'},
 {'x': 2, 'y': 5, 'w': 1, 'h': 1, 'content': 'AAD'},
 {'x': 3, 'y': 0, 'w': 1, 'h': 1, 'content': ''},
 {'x': 3,

In [21]:
for tab in tables: 
    html_table = tab.metadata.text_as_html
    table = BeautifulSoup(html_table, "html.parser")
    print(html_table)
    head = table.find('thead').find('tr').find_all('th')
    head = [col.text.strip() for col in head ]
    # print(head)
    rows = table.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele if ele else [] for ele in cols ]) # Get rid of empty values
    print(data)

<table><thead><tr><th>11</th><th>Hạng mục</th><th>Đơn vi do</th><th>Yêu cầu</th></tr></thead><tbody><tr><td>1</td><td>Thiết bị bay không người lái co bản</td><td></td><td></td></tr><tr><td>1</td><td>Hãng sản xuất</td><td></td><td>Ghi rõ</td></tr><tr><td>2</td><td>Nước sản xuất</td><td></td><td>Ghi rõ</td></tr><tr><td>3</td><td>Mã hiệu</td><td></td><td>Ghi rõ</td></tr><tr><td>4</td><td>Website của nhà sản xuất</td><td></td><td>Ghi rõ</td></tr><tr><td>5</td><td>Trọng lượng</td><td></td><td></td></tr><tr><td>5.1</td><td>Trọng lượng cất cánh (Không bao gồm camera)</td><td>Kg</td><td>516</td></tr><tr><td>5.2</td><td>Trong lương cất cánh tối đa</td><td>Kg</td><td>531</td></tr><tr><td>6</td><td>Chủng loại thiết bị</td><td></td><td>Drone công nghiệp</td></tr><tr><td>6.1</td><td>Kích thước thiết bị (Không bao gồm cánh quạt)</td><td></td><td>Là loại thiết bị nhỏ để gọn thuận tiện sử dụng trong điều kiện vận hành phức tap</td></tr><tr><td>6.2</td><td>Kích thước lớn nhất của thiết bị (Dài x Rộng x

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
def page_processor(elements):
    data = []
    table_elements_coords = [el.metadata.coordinates for el in elements if el.category == "Table"]
    print(table_elements_coords)
    # bboxes1_is_almost_subregion_of_bboxes2
    # Concatenate continual elements 
    for el in elements:
        if  len(data) > 0:
            last = data[-1]
            if (el.category[-4:] == "Text" and el.text[0].islower()) and last.category not in ["PageBreak", "Footer", "Table", "Image", "FigureCaption"]:
                data[-1].text += " " + el.text.strip()
                if data[-1].metadata.ocr_text:
                    concat_str = f" {last.metadata.ocr_text}" if last.metadata.ocr_text else ""
                    data[-1].metadata.ocr_text += concat_str
            elif (el.category == "Table" and last.category == "Table"):
                data[-1].metadata.text_as_html += el.metadata.text_as_html
                data[-1].text += " " + el.text.strip()
                if data[-1].metadata.ocr_text:
                    concat_str = f" {last.metadata.ocr_text}" if last.metadata.ocr_text else ""
                    data[-1].metadata.ocr_text += concat_str
            else:
                data.append(el)
            # is_page_break = False
        else:
            data.append(el)
    return data

processed = page_processor(elements)
print(len(elements) - len(processed), "elements removed due to page breaks")

[CoordinatesMetadata(points=((np.float64(278.2743225097656), np.float64(472.5968933105469)), (np.float64(278.2743225097656), np.float64(2947.802734375)), (np.float64(2315.127197265625), np.float64(2947.802734375)), (np.float64(2315.127197265625), np.float64(472.5968933105469))), system=<unstructured.documents.coordinates.PixelSpace object at 0x7f74315f5300>), CoordinatesMetadata(points=((np.float64(263.00634765625), np.float64(207.7759552001953)), (np.float64(263.00634765625), np.float64(2641.2568359375)), (np.float64(2345.567138671875), np.float64(2641.2568359375)), (np.float64(2345.567138671875), np.float64(207.7759552001953))), system=<unstructured.documents.coordinates.PixelSpace object at 0x7f7431434940>), CoordinatesMetadata(points=((np.float64(256.16217041015625), np.float64(199.45077514648438)), (np.float64(256.16217041015625), np.float64(2633.710205078125)), (np.float64(2344.855224609375), np.float64(2633.710205078125)), (np.float64(2344.855224609375), np.float64(199.450775146

In [ ]:
types = dict()
for el in elements:
    if el.category not in types:
        types[el.category] = list()
    types[el.category].append(el)

for key, value in types.items():
    print(key + ":")
    print("\n".join([el.text for el in value]))

Title:
TỔNG CÔNG TY ĐIỆN LỰC TP HÀ NỘI BAN KỸ THUẬT
CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc
TỜ TRÌNH Về việc phê duyệt tiêu chuẩn kỹ thuật thiết bị bay không người lái (UAV)
2. Yêu cầu kỹ thuật chung
3. Đặc tính kỹ thuật và cam kết
Ý kiến phê duyệt của Phó Tổng Giám đốc Nguyễn Anh Dũng
Phụ lục I: YÊU CẦU ĐẶC TÍNH KỸ THUẬT THIẾT BỊ BAY KHÔNG NGƯỜI LÁI
YÊU CẦU KỸ THUẬT CỦA BỘ CAMERA LIDAR
UncategorizedText:
Số: /KT
Hà Nội, ngày tháng năm 2025
a. Thiết bị bay:
2
2
3
3
f bộ nhớ lưu trữ:
4
4
Trân trọng./
Nơi nhận:
Nơi nhận:
Đào Ngọc Chung
Đào Ngọc Chung
5
5
7.11 tối đa 7.12 7.13 kháng gió tối đa m/s m/s m/s m/s m/s m/s m/s m/s
m/s m/s m/s m/s
m/s
m/s
m/s
m/s
215 25 25 2 12
≥ 15
≥ 12
Độ chênh lệch theo phương ngang khi bay
7.15 giữ độ cao (khi có RTK) giữ độ cao (khi có RTK) m m
giữ độ cao (khi có RTK)
m
m
Ghi rõ
Ghi rõ
6
Hãng sản xuất 2 Nước sản xuất 3
Nước sản xuất
3 Mã hiệu
Ghi rõ Ghi rõ Ghi rõ
Ghi rõ
Ghi rõ
Ghi rõ
Tích hợp hoặc đi kèm màn hình cảm ứng màu, để hiển 

# Do some overall statistics

In [ ]:
# from copy import deepcopy
# import os
# data = []
# strategy = "hi_res"
# model_name = "yolox" 
# def crawl_dataset(path):
#     if os.path.isdir(path):
#         for file in os.listdir(path):
#             crawl_dataset(os.path.join(path, file))
#     else:
#         if not path.endswith(".ini") and not path.endswith(".doc"):
#             print("Processing file:", path)
#             elements = partition(
#                 filename=path, 
#                 strategy=strategy, 
#                 languages=["vie", "eng"],
#                 infer_table_structure=True,
#                 skip_infer_table_types = [],
#                 hi_res_model_name=model_name
#             )
#             data.extend(deepcopy(elements))

# crawl_dataset("data/TCKT upload")

In [ ]:
# types = dict()
# for el in data:
#     if el.category not in types:
#         types[el.category] = list()
#     types[el.category].append(el)

# for key, value in types.items():
#     print(key + ":")
#     print("\n".join([el.text for el in value]))

In [ ]:
# for key, value in types.items():
#     print(f"{key}: {len(value)} elements")

The category shit only works on well-composed documents, else are trashed

# Testing manual filtering and formatting

In [ ]:
hierarchy_regex = {r"(?i)\b(?:chương|phụ\s*lục|phần)\s+(?:\d+|[ivx]+)(?:\.(?:\d+|[ivx]+))*[:\.]?": 1,
                   r"\bmục\s+(?:\d+|[ivx]+)(?:\.(?:\d+|[ivx]+))*[:\.]?": 2,
                   r"\b(?:điều|khoản)\s+\d+(?:\.(?:\d+|[ivx]+))*[:\.]?": 3,
                   r'^([ivx]+)(?:\.(?:\d+|[ivx]+))*\.': 4,
                   r"^(\d+)(?:\.(?:\d+))*\.": 5,
                   r'^[a-z]+[\.\)/]': 6,
                   r"^[-]": 7,
                   r"^[+]": 8}
roman_pattern = re.compile(r"""
    (XC|XL|L?X{0,3})     # Tens - 90 (XC), 40 (XL), 0-30 (0 to 3 X's), or 50-80 (L plus 0 to 3 X's)
    (IX|IV|V?I{0,3})     # Ones - 9 (IX), 4 (IV), 0-3 (I's), or 5-8 (V plus 0 to 3 I's)
    $""", re.VERBOSE)

In [ ]:
re.match(r"\bmục\s+(?:\d+|[ivxl]+)(?:\.(?:\d+|[ivx]+))*^[:\.]?", "mục lục")
#ivdx

In [ ]:
def parse_content(title, content):
    return [f"({t}) {c}" for t, c in zip(title, content) if len(c) > 0]

def roman_to_int(roman):
    roman_values = {
        'I': 1, 'V': 5, 'X': 10
    }

    total = 0
    prev_value = 0

    for char in reversed(roman.upper()):
        value = roman_values.get(char, 0)
        if value < prev_value:
            total -= value
        else:
            total += value
            prev_value = value

    return total

def character_to_int(str):
    return sum([(ord(char) - ord('a') + 1) * (1 + (len(str) - index - 1) * 26) for index, char in enumerate(str.lower())])

def numel_to_int(str):
    return int(str)

def to_int(str):
    
    # print(f"Converting: ({str})")
    if roman_pattern.match(str.upper()):
        return roman_to_int(str)
    elif re.match(r"^\d+", str):
        return numel_to_int(str)
    elif re.match(r"^[a-z]", str):
        return character_to_int(str)
    else:
        return [-1] # Negative value indicates an invalid input

def numbering(text):
    """
    Convert a string to an integer based on its numbering system.
    Supports Roman numerals, Arabic numerals, and lowercase letters.
    """
    return [to_int(part) for part in re.split(r"[:.\s]+", text) if part.strip() != '']


In [ ]:
def table_depth(element):
    return (-10, 0)

In [ ]:
def text_depth(element):
    """
    Calculate the depth of the text content based on its structure.
    """
    content = element.text.strip().strip("\"").lower()
    if not content:
        return (11, [-1])
    for pattern, order in hierarchy_regex.items():
        matched = re.match(pattern, content)
        if matched:
            if order <= 6:
                return (order, numbering(matched.group().replace("  " , " ").split()[-1]))
            else:
                return (order, [0])
    return (10, [0])  # Default case if no pattern matches

In [ ]:
def element_depth(element):
    if element.category in ['Image', 'FigureCaption', 'Page', 'PageBreak', 'Footer']:
        return 11
    elif element.category.endswith("Text"):
        return text_depth(element)
    elif element.category == "Table":
        return table_depth(element)
    else:
        return text_depth(element)

In [ ]:
for el in elements:
    if el.category[-4:] != "Table":
        el.metadata.text = clean_text(el.text)
        el.metadata.depth = text_depth(el)
    print(element_depth(el), el.category, el.metadata.category_depth, el.text)
    

(10, [0]) Title None TỔNG CÔNG TY ĐIỆN LỰC TP HÀ NỘI BAN KỸ THUẬT
(10, [0]) Title None CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc
(10, [0]) UncategorizedText None Số: /KT
(10, [0]) UncategorizedText None Hà Nội, ngày tháng năm 2025
(10, [0]) Title None TỜ TRÌNH Về việc phê duyệt tiêu chuẩn kỹ thuật thiết bị bay không người lái (UAV)
(10, [0]) NarrativeText None Kính gửi: Ông Phó Tổng Giám đốc Kỹ thuật
(10, [0]) NarrativeText None Căn cứ vào Quyết định số 8594/QĐ-EVNHANOI ngày 06/12/2023 Tổng Công ty Điện lực TP Hà Nội về việc giao danh mục mua sắm tài sản năm 2024 đợt 2 cho Công ty Lưới điện Cao thế thành phố Hà Nội;
(10, [0]) NarrativeText None Ban Kỹ thuật kính trình Tổng Giám đốc phê duyệt tiêu chuẩn kỹ thuật thiết bị bay không người lái công nghiệp (UAV) để tổ chức thực hiện mua sắm bổ sung trang bị cho Công ty Lưới điện cao thế TP Hà Nội với các nội dung sau: 1. Các yêu cầu chung về nhà sản xuất
(7, [0]) ListItem None - Nhà sản xuất phải có chứng nhận quản lý c

In [ ]:
class DocumentNode:
    def __init__(self, depth, path = "", element = None, father = None, prefix = ""):
        self.depth = depth
        self.father = father
        self.children = defaultdict(list)
        self.element = element
        self.prefix = prefix
        self.content = clean_text(element.text) if element else ""
        self.path = element.metadata.filename if element else path
    
    def set_father(self, father):
        if not self.check_father(father):
            return False
        depth, lev = self.depth
        # Get last elements since father is indicated beforehand
        lev = lev[-1] if isinstance(lev, list) else lev
        self.father = father
        father.children[depth].append(self)
        if lev == 0:
            lev = len(self.father.children[depth])
            self.depth = (depth, [lev])
        self.path = father.path + "/" + f"{self.prefix}{depth}.{lev}"
        return True

    def check_father(self, node):
        """
        Check if the element is a child of this node.
        """
        n_depth, n_lev = node.depth
        s_depth, s_lev = self.depth
        if n_depth > s_depth or n_depth < 0:
            return False
        elif (n_depth == s_depth) and (len(s_lev) - len(n_lev) == 1):
            for s_l, n_l in zip(s_lev[:-1], n_lev):
                if s_l != n_l:
                    return False
            if s_depth not in node.children.keys() and s_lev[-1] > 1:
                return False
            # Add tolerance for falsely partitioning
            elif s_lev[-1] > len(node.children[s_depth]) + 2: 
                return False
            return True
        elif (n_depth < s_depth):
            if s_depth not in node.children.keys() and s_lev[-1] > 1:
                return False
            # Add tolerance for falsely partitioning
            elif s_lev[-1] > len(node.children[s_depth]) + 2: 
                return False
            return True
        else: 
            return False
            
    def __repr__(self):
        return f"DocumentNode(depth={self.depth}, path={self.path}, content={self.content[:30]})"
    
            

In [ ]:
root = DocumentNode(depth=(0, [0]), path="")

In [ ]:
document_stack = [root]
tabled = True
for el in processed:
    if el.category[-4:] != "Table":
        depth = text_depth(el)
        if depth[0] == 11:
            continue
        elif depth[0] == 10 and el.category == "ListItem":
            depth = (6, [0])
        el_node = DocumentNode(depth, element=el, father=None)
        for node in document_stack[::-1]:
            if el_node.set_father(node):
                break
        document_stack.append(el_node)

In [ ]:
for node in document_stack:
    print(node.depth, node.path, node.element.text if node.element else "")

(0, [0])  
(10, [1]) /10.1 TỔNG CÔNG TY ĐIỆN LỰC TP HÀ NỘI BAN KỸ THUẬT
(10, [2]) /10.2 CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc
(10, [3]) /10.3 Số: /KT
(10, [4]) /10.4 Hà Nội, ngày tháng năm 2025
(10, [5]) /10.5 TỜ TRÌNH Về việc phê duyệt tiêu chuẩn kỹ thuật thiết bị bay không người lái (UAV)
(10, [6]) /10.6 Kính gửi: Ông Phó Tổng Giám đốc Kỹ thuật
(10, [7]) /10.7 Căn cứ vào Quyết định số 8594/QĐ-EVNHANOI ngày 06/12/2023 Tổng Công ty Điện lực TP Hà Nội về việc giao danh mục mua sắm tài sản năm 2024 đợt 2 cho Công ty Lưới điện Cao thế thành phố Hà Nội;
(10, [8]) /10.8 Ban Kỹ thuật kính trình Tổng Giám đốc phê duyệt tiêu chuẩn kỹ thuật thiết bị bay không người lái công nghiệp (UAV) để tổ chức thực hiện mua sắm bổ sung trang bị cho Công ty Lưới điện cao thế TP Hà Nội với các nội dung sau: 1. Các yêu cầu chung về nhà sản xuất
(7, [1]) /7.1 - Nhà sản xuất phải có chứng nhận quản lý chất lượng ISO hoặc tương đương.
(7, [2]) /7.2 - Nhà thầu phải cung cấp đầy đủ thông ti